In [1]:
import polars as pl
from db.model import Tactics, Techniques, TacTech, TacticOrders, Datasources, TechData, DataCollection
from miter.create_db import create_teqnique_datasource_relation_records, save_to_mysql, load_datasource_as_records
DataCollection.create_table()

In [2]:
COLUMN_MAPPING_DATASOURCE = {    
    "name": "datasource",
    "ID": "datasource_id",
    "STIX ID": "stix_id",
    "description": "description_en",
    # "platforms": "platforms",
    "created": "created",
    "modified": "last_modified",
    "type": "type",
    "version": "version",
    "url": "url"
}

In [12]:
df = pl.read_excel("/home/work/data/enterprise-attack-v15.1-datasources.xlsx")
df = df.rename(COLUMN_MAPPING_DATASOURCE).drop_nulls("collection layers")
df = df.with_columns(pl.col("collection layers").str.split(",")).explode("collection layers").rename({"collection layers":"collection_layer"})
df = df.with_columns(pl.col("collection_layer").str.strip_chars(" ")).select(["datasource_id", "collection_layer"])
df = df.with_row_count("id")
df = df.with_columns((pl.col("id") + 1).alias("id"))
data = df.to_dicts()

/tmp/ipykernel_755228/1281368422.py:1: DeprecationWarning: `with_row_count` is deprecated. Use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  df = df.with_row_count("id")


In [16]:
save_to_mysql(DataCollection, data)